# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-04 08:44:11] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-04 08:44:11] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-04 08:44:11] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-04 08:44:13] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-04 08:44:13] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=75.34 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=88 avail_mem=74.72 GB):  20%|██        | 4/20 [00:00<00:01, 13.16it/s]

Capturing batches (bs=48 avail_mem=74.70 GB):  50%|█████     | 10/20 [00:00<00:00, 19.81it/s]

Capturing batches (bs=16 avail_mem=74.62 GB):  65%|██████▌   | 13/20 [00:00<00:00, 21.01it/s]

Capturing batches (bs=1 avail_mem=74.59 GB): 100%|██████████| 20/20 [00:01<00:00, 19.37it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Yana Feigelman and I have been an academic instructor in a university teaching Chinese and English. I've been teaching at this university for four years. I have a Bachelor's degree in Computer Science from the Hebrew University of Jerusalem and a Master's degree in English teaching from Columbia University. I have taught in China for the past year. I started teaching English as a second language at the university 2 years ago. 

I am an avid reader of all languages and I love reading about the world. I enjoy writing my blog post on my own, and I love how much I learn from these posts. I recently published my first blog
Prompt: The president of the United States is
Generated text:  currently 32 years older than the president of the United States 50 years ago. If the president of the United States 50 years ago was 32 years old, what is the current age of the president of the United States? Let's denote the current age of the president of the Unit

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm a [insert your profession or role here]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Insert your personal information here, such as your age, gender, occupation, and any other relevant details]. I'm always looking for new challenges and opportunities to grow and learn. What are some of your favorite things to do? [Insert your favorite activities or hobbies here, such as reading, playing sports, or spending time with friends and family]. I'm always looking for ways to improve my skills and

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a major cultural and economic center, with a rich history dating back to the Roman Empire and the French Revolution. Paris is home to many famous French artists, writers, and musicians, and is a popular tourist destination for its beautiful architecture, vibrant nightlife, and annual festivals. The city is also home to the French Parliament and the French Parliament of the European Union. Paris is a major hub for international business and trade, and is a popular destination for tourists from around the world. The city

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends:

1. Increased focus on ethical AI: As more people become aware of the potential risks of AI, there will be a greater emphasis on developing AI that is designed to be ethical and responsible. This could involve developing AI that is transparent, accountable, and accountable to human values.

2. Greater use of AI in healthcare: AI is already being used in healthcare to help diagnose and treat diseases, and there is a growing expectation that it will become even more integrated into the healthcare system in the future. This



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I am a [Your profession] with a strong passion for [Your area of interest or hobby]. I enjoy [What excites you about your career], and I believe that [How you believe you can make the world a better place]. I believe that my skills and experience make me a valuable asset to [Your company name] and I am looking forward to [Your first introduction/coming to you]! If you would like to meet me, please feel free to introduce yourself! #selfintroduction #newbie #entrepreneur #career #careerfocus #careergoals #selfimaginings #empathy

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is known for its iconic landmarks, such as Notre-Dame Cathedral and the Eiffel Tower, and its rich history dating back over 2000 years. Paris is also known for its delicious cuisine, fashion, and art scene. The city i

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 and

 security

 that

 will

 require

In [6]:
llm.shutdown()